In [1]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
data_size = 1000
step_size = 20
split_rate = 0.7

dataset = []
for i in range(data_size):
    d = np.random.randint(2,size=step_size)
    idx = np.count_nonzero(d)
    d = np.append(d,np.identity(21)[idx:idx+1])
    dataset.append(d)
dataset = np.array(dataset)

train_dataset = dataset[:int(split_rate*data_size)]
test_dataset = dataset[:int(split_rate*data_size)]

train_x_dataset = train_dataset[:,:-21].reshape(-1,step_size,1)
train_y_dataset = train_dataset[:,-21:]
test_x_dataset = test_dataset[:,:-21].reshape(-1,step_size,1)
test_y_dataset = test_dataset[:,-21:]

In [3]:
tf.reset_default_graph()

num_hidden = 24
x = tf.placeholder(tf.float32,[None,step_size,1])
y = tf.placeholder(tf.float32,[None,21])
cell = tf.contrib.rnn.LSTMCell(num_hidden)
val, state = tf.nn.dynamic_rnn(cell,x,dtype=tf.float32)
val = tf.transpose(val,[1,0,2])
last = tf.gather(val,int(val.get_shape()[0])-1)

w = tf.Variable(tf.random_normal([num_hidden,21]),dtype=tf.float32)
b = tf.Variable(tf.constant(0.0),dtype=tf.float32)
o = tf.nn.softmax(tf.matmul(last,w) + b)
pred = tf.argmax(o,1)

cost = tf.reduce_mean(tf.square(-y*tf.log(o)))
grad = tf.train.AdamOptimizer(0.1).minimize(cost)
acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y,1),pred),dtype=tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        c,_ = sess.run([cost,grad],{x:train_x_dataset,y:train_y_dataset})
        if i % 100 == 0:
            a = sess.run(acc,{x:test_x_dataset,y:test_y_dataset})
            print a
            
    test_x = np.random.randint(2,size=step_size)
    count = np.count_nonzero(test_x)
    test_y = np.identity(21)[count:count+1]
    print 'count = ' ,count
    print 'predicted count = ', sess.run(pred,{x:test_x.reshape(1,step_size,1)})[0]

0.174286
0.575714
0.977143
0.205714
0.964286
0.992857
1.0
1.0
1.0
1.0
count =  11
predicted count =  11


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
